In [3019]:
import random

# StringIO behaves like a file object 
from io import StringIO 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.datasets import load_boston
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf

import copy


from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


%matplotlib inline 

In [3020]:
def median(a,b,c):
    if a > b:
        if a < c:
            return a
        elif b > c:
            return b
        else:
            return c
    else:
        if a > c:
            return a
        elif b < c:
            return b
        else:
            return c

In [3021]:
class CqkProblem:
    def __init__(self, r, n, d, a, b, low, up):
        self.n = n
        self.r = r
        self.d = list(d)
        self.a = list(a)
        self.b = list(b)
        self.low = list(low)
        self.up = list(up)

In [3022]:
def generate_cqk_problem(n):
    d = []
    low = []
    up = []
    b = []
    a = []
    temp = 0
    lb = 0.0
    ub = 0.0
    lower = 10
    upper = 25
    r = 0

    for i in range(n):
        
        b.append(10 + 14*random.random())
        low.append(1 + 14*random.random())
        up.append(1 + 14*random.random())
        if low[i] > up[i]:
            temp = low[i]
            low[i] = up[i]
            up[i] = temp
        
        lb = lb + b[i]*low[i];
        ub = ub + b[i]*up[i];
        
        #Uncorrelated
        d.append(random.randint(10,25))
        a.append(random.randint(10,25))
        
    r = lb + (ub - lb)*0.7;
    
    return CqkProblem( r, n, d, a, b, low, up)

In [3023]:

def initial_lambda(p, lamb):
    s0=0.0
    q0=0.0
    slopes = []
    for i in range(p.n):
        slopes.append((p.b[i]/p.d[i])*p.b[i])
        s0 = s0 + (p.a[i] * p.b[i]) / p.d[i]
        q0 = q0 + (p.b[i] * p.b[i]) / p.d[i]
    lamb = (p.r-s0)/q0
    return lamb, slopes

In [3024]:
def phi_lambda(p,lamb,phi,deriv,slopes,r):
    deriv = 0.0
    phi = r * -1
    x = []
    
    for i in range(p.n):
        
        x.append( (p.b[i] * lamb + p.a[i])/p.d[i])

        if x[i] < p.low[i]:
            x[i] = p.low[i]
        elif x[i] > p.up[i]:
            x[i] = p.up[i]
        else:
            deriv = deriv + slopes[i];
        phi = phi + p.b[i] * x[i];
    return deriv, phi, x

In [3025]:
MAX_IT = 10
INFINITO_NEGATIVO = -999999999;
INFINITO_POSITIVO = 999999999;

def newton(p):
    lambs = [] 
    derivs = []
    phis = []
    phi = 0
    lamb = 0
    alfa = INFINITO_NEGATIVO;
    beta = INFINITO_POSITIVO;
    phi_alfa = 0.0;
    phi_beta = 0.0;
    deriv = 0
    x = []
    r = p.r
    
    lamb, slopes = initial_lambda(p,lamb)
    deriv, phi, x = phi_lambda(p,lamb,phi,deriv,slopes,r)
    lambs.append(lamb)
    derivs.append(deriv)
    phis.append(phi)
    
    it = 1
    
    while phi != 0.0 and it <= MAX_IT:
        if phi > 0:
#             print("positivo")
            beta = lamb
            lambda_n = 0.0
            if deriv > 0.0:
                
                lambda_n = lamb - (phi/deriv)
                if abs(lambda_n - lamb) <= 0.00000000001:
                    phi = 0.0
                    break
                if lambda_n > alfa:
                    lamb = lambda_n
                else:
#                     print("aqui")
                    phi_beta = phi;
#                     lamb = secant(p,x,alfa,beta,phi_alfa,phi_beta,r);
#             if deriv == 0.0:
#                 lamb = breakpoint_to_the_left(p,lamb);
#                 if lamb <= INFINITO_NEGATIVO or lamb >= INFINITO_POSITIVO:
#                     break
                
        else:
            if it == 1:
                negativo = True
#             print("negativo")
            alfa = lamb;
            lambda_n = 0.0;

            if deriv > 0.0:
                lambda_n = lamb - (phi/deriv)
                if abs(lambda_n - lamb) <= 0.00000000001:
                    phi = 0.0
                    break
                
                if lambda_n < beta:
                    lamb = lambda_n
                else:
#                     print("aqui")
                    phi_alfa = phi;
#                     lamb = secant(p,x,alfa,beta,phi_alfa,phi_beta,r);
#             if deriv == 0.0:
#                 lamb = breakpoint_to_the_right(p,lamb)
#                 if lamb <= INFINITO_NEGATIVO or lamb >= INFINITO_POSITIVO:
#                     break
        
        
        deriv, phi, x = phi_lambda(p,lamb,phi,deriv,slopes,r)
        it = it + 1
        lambs.append(lamb)
        derivs.append(deriv)
        phis.append(phi)
        
    if phi == 0.0:
        return it,lambs,derivs,phis
    elif alfa == beta:
        return -1,lambs,derivs,phis
    else:
        return -2,lambs,derivs,phis

In [3026]:
def func_ty(p):
    t = np.arange(0, 15, 0.5).tolist()
    y = copy.deepcopy(t)
    for j in range(len(t)):
        lamb = t[j]

        soma = 0

        for i in range(n):
            soma = soma + p.b[i] * median(p.low[i], (p.b[i] *lamb + p.a[i]) / p.d[i], p.up[i])

        y[j] = soma
    return y

# Geramos modelo para lambda inicial

In [3118]:
lista = []
for i in range(5000):
    n = 100
    p = generate_cqk_problem(n)
    it,lambs, derivs,phis = newton(p)
    soma_a = 0
    soma_b = 0
    soma_low = 0
    soma_d = 0
    soma_up = 0
    for i in range(n):
        soma_a += p.a[i]
        soma_b += p.b[i]
        soma_low += p.low[i]
        soma_d += p.d[i]
        soma_up += p.up[i]
    
    soma_a = soma_a/n
    soma_b = soma_b/n
    soma_low = soma_low/n
    soma_d = soma_d/n
    soma_up = soma_up/n
    r = p.r/n
    deriv = derivs[0]/n
#     l_rs = [soma_a, soma_b, soma_low, soma_up, soma_d, p.r, lambs[0], lambs[-1]]
    if it > 3 and it < 6:
        l_rs = [soma_a, soma_b, soma_d, r,lambs[0]]
#         l_rs = [p.a[0],p.a[1],p.a[2], p.b[0], p.b[1], p.b[2],p.d[0],p.d[1],p.d[2],p.r,lambs[0],derivs[0],phis[0],lambs[1],lambs[-1]]

#         l_rs = [p.a[0],p.a[1],p.a[2], p.b[0], p.b[1], p.b[2],p.low[0],p.low[1],p.low[2],p.d[0],p.d[1],p.d[2],p.up[0],p.up[1],p.up[2],p.r,lambs[0],lambs[1],lambs[-1]]

        lista.append(l_rs)
        if len(lista) == 1000:
            break

In [3119]:
np.savetxt('instance_test.txt', lista, delimiter = ' ',newline='\n', fmt="%f")

In [3120]:
c = ''
with open("instance_test.txt", "r") as fd:
    c = StringIO(fd.read())

In [3121]:
d = c.read()
c = StringIO(d) 
d = np.loadtxt(c) 
feature_names = ['a','b','d','r','inicital_lamb']


# feature_names = ['a','b','d','r','inicital_lamb','init_phi','init_deriv','second_lambda','third_lambda','fourth_lambda','final_lamb']
# feature_names = ['a1','a2','a3', 'b1','b2','b3', 'd1','d2','d3', 'r','inicital_lamb','init_deriv','init_phi','second_lambda','final_lamb']
# feature_names = ['a1','a2','a3', 'b1','b2','b3', 'low1','low2','low3', 'd1','d2','d3','up1','up2','up3', 'r','inicital_lamb','second_lambda', 'final_lamb']

In [3122]:
knapsack = {"data":d, "feature_names": feature_names}
dataset = pd.DataFrame(knapsack['data'], columns = knapsack['feature_names'])

In [3123]:
# Coletando x e y

X = dataset.iloc[:,:-1]
y = dataset['inicital_lamb'].values

In [3124]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [3125]:
# Padronização
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [3126]:
# Aplicando a padronização aos dados
X_train_p = scaler.transform(X_train)
X_test_p = scaler.transform(X_test)

In [3127]:
# from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor

In [3128]:
# mlp = MLPClassifier(hidden_layer_sizes = (30,30,30))
regr = MLPRegressor(activation='logistic',random_state=1, max_iter=5000).fit(X_train_p, y_train)

In [3129]:
regr.fit(X_train_p, y_train)

MLPRegressor(activation='logistic', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=5000,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=1, shuffle=True, solver='adam',
             tol=0.0001, validation_fraction=0.1, verbose=False,
             warm_start=False)

In [3130]:
regr.predict(X_test_p[:5])

array([6.22748911, 6.1094987 , 7.30196276, 6.81242267, 6.8591698 ])

In [3131]:
regr.score(X_test_p, y_test)

0.9247306630895714

# Geramos modelo para segundo lambda


In [3204]:
def initial_lambda_regr(p):
    
    soma_a = 0
    soma_b = 0
    soma_d = 0
    slopes = []
    for i in range(n):
        soma_a += p.a[i]
        soma_b += p.b[i]
        soma_d += p.d[i]
        slopes.append((p.b[i]/p.d[i])*p.b[i])

    soma_a = soma_a/n
    soma_b = soma_b/n
    soma_d = soma_d/n
    r = p.r/n
    lamb = (p.r-s0)/q0
    return lamb, slopes
#     return regr.predict(scaler.transform(np.asarray([[soma_a,soma_b,soma_d,r]]))),slopes


In [3205]:
def phi_lambda_model(p,lamb,phi,deriv,slopes,r):
    deriv = 0.0
    phi = r * -1
    x = []
    
    for i in range(p.n):
        
        x.append( (p.b[i] * lamb + p.a[i])/p.d[i])

        if x[i] < p.low[i]:
            x[i] = p.low[i]
        elif x[i] > p.up[i]:
            x[i] = p.up[i]
        else:
            deriv = deriv + slopes[i];
        phi = phi + p.b[i] * x[i];
    return deriv, phi, x

In [3206]:
MAX_IT = 10
INFINITO_NEGATIVO = -999999999;
INFINITO_POSITIVO = 999999999;

def newton_primeiro_lambda(p):
    lambs = [] 
    derivs = []
    phis = []
    phi = 0
    lamb = 0
    alfa = INFINITO_NEGATIVO;
    beta = INFINITO_POSITIVO;
    phi_alfa = 0.0;
    phi_beta = 0.0;
    deriv = 0
    x = []
    r = p.r
    
    lamb, slopes = initial_lambda(p,lamb)
    deriv, phi, x = phi_lambda_model(p,lamb,phi,deriv,slopes,r)
    lambs.append(lamb)
    derivs.append(deriv)
    phis.append(phi)
    
    it = 1
    
    while phi != 0.0 and it <= MAX_IT:
        if phi > 0:
#             print("positivo")
            beta = lamb
            lambda_n = 0.0
            if deriv > 0.0:
                
                lambda_n = lamb - (phi/deriv)
                if abs(lambda_n - lamb) <= 0.00000000001:
                    phi = 0.0
                    break
                if lambda_n > alfa:
                    lamb = lambda_n
                else:
#                     print("aqui")
                    phi_beta = phi;
#                     lamb = secant(p,x,alfa,beta,phi_alfa,phi_beta,r);
#             if deriv == 0.0:
#                 lamb = breakpoint_to_the_left(p,lamb);
#                 if lamb <= INFINITO_NEGATIVO or lamb >= INFINITO_POSITIVO:
#                     break
                
        else:
            if it == 1:
                negativo = True
#             print("negativo")
            alfa = lamb;
            lambda_n = 0.0;

            if deriv > 0.0:
                lambda_n = lamb - (phi/deriv)
                if abs(lambda_n - lamb) <= 0.00000000001:
                    phi = 0.0
                    break
                
                if lambda_n < beta:
                    lamb = lambda_n
                else:
#                     print("aqui")
                    phi_alfa = phi;
#                     lamb = secant(p,x,alfa,beta,phi_alfa,phi_beta,r);
#             if deriv == 0.0:
#                 lamb = breakpoint_to_the_right(p,lamb)
#                 if lamb <= INFINITO_NEGATIVO or lamb >= INFINITO_POSITIVO:
#                     break
        
        
        deriv, phi, x = phi_lambda(p,lamb,phi,deriv,slopes,r)
        it = it + 1
        lambs.append(lamb)
        derivs.append(deriv)
        phis.append(phi)
        
    if phi == 0.0:
        return it,lambs,derivs,phis
    elif alfa == beta:
        return -1,lambs,derivs,phis
    else:
        return -2,lambs,derivs,phis

In [3207]:
lista = []
for i in range(20000):
    n = 100
    p = generate_cqk_problem(n)
    it,lambs,derivs,phis = newton_primeiro_lambda(p)
    soma_a = 0
    soma_b = 0
    soma_d = 0
    for i in range(n):
        soma_a += p.a[i]
        soma_b += p.b[i]
        soma_d += p.d[i]
    
    soma_a = soma_a/n
    soma_b = soma_b/n
    soma_d = soma_d/n
    r = p.r/n
#     l_rs = [soma_a, soma_b, soma_low, soma_up, soma_d, p.r, lambs[0], lambs[-1]]
    if it > 0 and it < 5:
        l_rs = [soma_a, soma_b, soma_d, r,lambs[0],phis[0],derivs[0],phis[1],lambs[1],lambs[-1]]

#         l_rs = [p.a[0],p.a[1],p.a[2], p.b[0], p.b[1], p.b[2],p.d[0],p.d[1],p.d[2],p.r,lambs[0],derivs[0],phis[0],lambs[1],lambs[-1]]

#         l_rs = [p.a[0],p.a[1],p.a[2], p.b[0], p.b[1], p.b[2],p.low[0],p.low[1],p.low[2],p.d[0],p.d[1],p.d[2],p.up[0],p.up[1],p.up[2],p.r,lambs[0],lambs[1],lambs[-1]]

        lista.append(l_rs)
        if len(lista) == 10000:
            break

In [3208]:
np.savetxt('instance_test.txt', lista, delimiter = ' ',newline='\n', fmt="%f")

In [3209]:
c = ''
with open("instance_test.txt", "r") as fd:
    c = StringIO(fd.read())

d = c.read()
c = StringIO(d) 
d = np.loadtxt(c) 
feature_names = ['a','b','d','r','inicital_lamb','phi_initial','deriv_initial','second_phi','second_lamb','final_lamb']   



In [3215]:
knapsack_second = {"data":d, "feature_names": feature_names}
dataset_second = pd.DataFrame(knapsack_second['data'], columns = knapsack_second['feature_names'])


# Coletando x e y

X_second = dataset_second.iloc[:,:-3]
y_second = dataset_second['second_lamb'].values

In [3291]:
dataset_second

,a,b,d,r,inicital_lamb,phi_initial,deriv_initial,second_phi,second_lamb,final_lamb
0,17.52,16.515474,17.53,156.078163,7.866380,-891.288454,761.860351,-38.823544,9.036264,9.095324
1,17.54,16.951509,16.61,152.801912,6.755693,-1207.598407,755.518723,21.007378,8.354063,8.316486
2,17.31,17.119538,17.05,142.245602,6.463221,-957.658400,776.926362,-87.361606,7.695845,7.845849
3,17.73,16.706896,17.41,144.512202,6.763771,-1041.331428,523.260341,-68.527982,8.753854,8.899579
4,17.10,17.138578,17.53,152.563926,7.094160,-1200.718662,702.072582,-87.235078,8.804409,8.973913
...,...,...,...,...,...,...,...,...,...,...
9995,17.33,17.138600,17.30,146.567360,6.564720,-1226.325975,596.733014,-92.700780,8.619787,8.782494
9996,16.62,16.667576,17.32,150.086351,7.323448,-1061.697094,732.720571,-184.042877,8.772428,9.131766
9997,18.11,17.071313,18.09,149.331722,7.128118,-1533.810382,768.802581,-111.905781,9.123182,9.310151
9998,17.62,16.863073,17.34,149.465798,6.846563,-1165.266232,639.700687,-91.226711,8.668143,8.860779


In [3217]:
X_train_second, X_test_second, y_train_second, y_test_second = train_test_split(X_second, y_second)

# Padronização
scaler_second = StandardScaler()
scaler_second.fit(X_train_second)

# Aplicando a padronização aos dados
X_train_p_second = scaler_second.transform(X_train_second)
X_test_p_second = scaler_second.transform(X_test_second)

In [3218]:
regr_second_lambda = MLPRegressor(activation='logistic',random_state=1, max_iter=5000).fit(X_train_p_second, y_train_second)

In [3219]:
regr_second_lambda.fit(X_train_p_second, y_train_second)

MLPRegressor(activation='logistic', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=5000,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=1, shuffle=True, solver='adam',
             tol=0.0001, validation_fraction=0.1, verbose=False,
             warm_start=False)

In [3284]:
regr_second_lambda.score(X_test_p_second, y_test_second)

0.9827192189904328

In [3285]:
# print(X_test_p_second[:1])
regr_second_lambda.predict(X_test_p_second[:10])

array([8.78111221, 6.61563923, 8.094837  , 9.05991923, 8.59636432,
       7.86643674, 7.82502797, 7.94006535, 9.24479003, 9.34992004])

In [3286]:
y_test_second[:10]

array([8.816307, 6.876359, 8.03368 , 9.008379, 8.586252, 7.804226,
       7.887314, 8.000323, 9.23473 , 9.365615])

# Predicao para lambda final

In [3287]:
def initial_lambda_regr_terceiro(p):
    s0=0.0
    q0=0.0
    
    soma_a = 0
    soma_b = 0
    soma_d = 0
    slopes = []
    for i in range(n):
        soma_a += p.a[i]
        soma_b += p.b[i]
        soma_d += p.d[i]
        slopes.append((p.b[i]/p.d[i])*p.b[i])
        s0 = s0 + (p.a[i] * p.b[i]) / p.d[i]
        q0 = q0 + (p.b[i] * p.b[i]) / p.d[i]

    soma_a = soma_a/n
    soma_b = soma_b/n
    soma_d = soma_d/n
    rn = p.r/n
    lamb = (p.r-s0)/q0

    return lamb, slopes,soma_a,soma_b,soma_d,rn
#     return regr.predict(scaler.transform(np.asarray([[soma_a,soma_b,soma_d,r]])))[0],slopes,soma_a,soma_b,soma_d,rn

In [3288]:
def phi_lambda_model(p,lamb,phi,deriv,slopes,r):
    deriv = 0.0
    phi = r * -1
    x = []
    
    for i in range(p.n):
        
        x.append( (p.b[i] * lamb + p.a[i])/p.d[i])

        if x[i] < p.low[i]:
            x[i] = p.low[i]
        elif x[i] > p.up[i]:
            x[i] = p.up[i]
        else:
            deriv = deriv + slopes[i];
        phi = phi + p.b[i] * x[i];
    return deriv, phi, x

In [3305]:
MAX_IT = 10
INFINITO_NEGATIVO = -999999999;
INFINITO_POSITIVO = 999999999;

def newton_segundo_lambda(p):
    lambs = [] 
    phis = []
    derivs = []
    phi = 0
    alfa = INFINITO_NEGATIVO;
    beta = INFINITO_POSITIVO;
    phi_alfa = 0.0;
    phi_beta = 0.0;
    x = []
    r = p.r
    deriv = 0
    slopes = []
    lamb = 0
    
    lamb,slopes,soma_a,soma_b,soma_d,rn = initial_lambda_regr_terceiro(p)
#     deriv, phi, x = phi_lambda_model(p,lamb,phi,deriv,slopes,r)

    lambs.append(lamb)
    
    
    test = regr_second_lambda.predict(scaler_second.transform(np.asarray([[soma_a,soma_b,soma_d,rn,lamb,phi,deriv]])))[0]
    print(test)
#     lambs.append(lamb)
    
#     lamb, slopes = initial_lambda(p,lamb)
    deriv, phi, x = phi_lambda_model(p,lamb,phi,deriv,slopes,r)
    derivs.append(deriv)
    phis.append(phi)
    
    
    it = 1
   
    while phi != 0.0 and it <= MAX_IT:
        if phi > 0:
            print("positivo")
            beta = lamb
            lambda_n = 0.0
            if deriv > 0.0:
                
                lambda_n = lamb - (phi/deriv)
                if abs(lambda_n - lamb) <= 0.00000000001:
                    phi = 0.0
                    break
                if lambda_n > alfa:
                    lamb = lambda_n
                else:
#                     print("aqui")
                    phi_beta = phi;
#                     lamb = secant(p,x,alfa,beta,phi_alfa,phi_beta,r);
#             if deriv == 0.0:
#                 lamb = breakpoint_to_the_left(p,lamb);
#                 if lamb <= INFINITO_NEGATIVO or lamb >= INFINITO_POSITIVO:
#                     break
                
        else:
            print("negativo")
            alfa = lamb;
            lambda_n = 0.0;

            if deriv > 0.0:
                lambda_n = lamb - (phi/deriv)
                if abs(lambda_n - lamb) <= 0.00000000001:
                    phi = 0.0
                    break
                
                if lambda_n < beta:
                    lamb = lambda_n
                else:
#                     print("aqui")
                    phi_alfa = phi;
#                     lamb = secant(p,x,alfa,beta,phi_alfa,phi_beta,r);
#             if deriv == 0.0:
#                 lamb = breakpoint_to_the_right(p,lamb)
#                 if lamb <= INFINITO_NEGATIVO or lamb >= INFINITO_POSITIVO:
#                     break
        
        
        deriv, phi, x = phi_lambda_model(p,lamb,phi,deriv,slopes,r)
        it = it + 1
        lambs.append(lamb)
        derivs.append(deriv)
        phis.append(phi)
        
    if phi == 0.0:
        return it,lambs,phis,derivs
    elif alfa == beta:
        return -1,lambs,phis,derivs
    else:
        return -2,lambs,phis,derivs
    



In [3306]:
lista = []
for i in range(5):
    n = 100
    p = generate_cqk_problem(n)
    it,lambs,phis,derivs = newton_segundo_lambda(p)
    print(it)
    soma_a = 0
    soma_b = 0
    soma_d = 0
    for i in range(n):
        soma_a += p.a[i]
        soma_b += p.b[i]
        soma_d += p.d[i]
    
    soma_a = soma_a/n
    soma_b = soma_b/n
    soma_d = soma_d/n
    r = p.r/n
#     l_rs = [soma_a, soma_b, soma_low, soma_up, soma_d, p.r, lambs[0], lambs[-1]]
    if it > 0 and it < 5:
        l_rs = [soma_a, soma_b, soma_d, r,lambs[0],phis[0],derivs[0],phis[1],lambs[1],lambs[-1]]
#         l_rs = [p.a[0],p.a[1],p.a[2], p.b[0], p.b[1], p.b[2],p.d[0],p.d[1],p.d[2],p.r,lambs[0],derivs[0],phis[0],lambs[1],lambs[-1]]

#         l_rs = [p.a[0],p.a[1],p.a[2], p.b[0], p.b[1], p.b[2],p.low[0],p.low[1],p.low[2],p.d[0],p.d[1],p.d[2],p.up[0],p.up[1],p.up[2],p.r,lambs[0],lambs[1],lambs[-1]]

        lista.append(l_rs)
        if len(lista) == 10000:
            break

8.533035951339594
negativo
negativo
positivo
3
8.479545428557918
negativo
negativo
negativo
negativo
4
7.875061014480469
negativo
negativo
negativo
negativo
4
8.52295401467521
negativo
negativo
positivo
negativo
4
8.368005374432595
negativo
negativo
negativo
negativo
4


In [3307]:
np.savetxt('instance_test.txt', lista, delimiter = ' ',newline='\n', fmt="%f")

In [3308]:
c = ''
with open("instance_test.txt", "r") as fd:
    c = StringIO(fd.read())

d = c.read()
c = StringIO(d) 
d = np.loadtxt(c) 
feature_names = ['a','b','d','r','inicital_lamb','phi_initial','deriv_initial','second_phi','second_lamb','final_lamb']   

knapsack_third = {"data":d, "feature_names": feature_names}
dataset_third = pd.DataFrame(knapsack_third['data'], columns = knapsack_third['feature_names'])


# Coletando x e y

X_third = dataset_third.iloc[:,:-1]
y_third = dataset_third['final_lamb'].values

In [3309]:
dataset_third

,a,b,d,r,inicital_lamb,phi_initial,deriv_initial,second_phi,second_lamb,final_lamb
0,17.76,17.025510,17.73,150.412081,6.969957,-1514.030800,717.533679,-89.631159,9.080006,9.229499
1,18.15,17.307834,16.86,158.776057,6.963048,-1143.907435,882.162495,-130.666025,8.259756,8.458947
2,18.03,17.109323,17.09,143.299188,6.298711,-964.922682,685.627402,-73.946997,7.706068,7.835231
3,17.40,16.965988,18.09,145.469193,6.964585,-991.683179,638.031011,-211.082107,8.518871,8.997597
4,18.37,16.667807,16.89,148.848228,6.808125,-1033.452220,827.793447,-81.466408,8.056567,8.177303


In [3089]:
X_train_third, X_test_third, y_train_third, y_test_third = train_test_split(X_third, y_third)

# Padronização
scaler_third = StandardScaler()
scaler_third.fit(X_train_third)

# Aplicando a padronização aos dados
X_train_p_third = scaler_third.transform(X_train_third)
X_test_p_third = scaler_third.transform(X_test_third)

In [3090]:
regr_third_lambda = MLPRegressor(activation='logistic',random_state=1, max_iter=5000).fit(X_train_p_third, y_train_third)

In [3091]:
regr_third_lambda.fit(X_train_p_third, y_train_third)

MLPRegressor(activation='logistic', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=5000,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=1, shuffle=True, solver='adam',
             tol=0.0001, validation_fraction=0.1, verbose=False,
             warm_start=False)

In [3092]:
regr_third_lambda.score(X_test_p_third, y_test_third)

0.993079224004893

In [3093]:
regr_third_lambda.predict(X_test_p_third[:20])

array([8.56861492, 9.33599095, 8.85081437, 8.63613154, 8.62460186,
       8.37241868, 7.92555192, 9.34852908, 8.65482519, 7.80956563,
       8.77641947, 9.60625718, 8.55126625, 9.14596024, 8.76606819,
       8.83954137, 9.03676938, 9.53639663, 8.35275554, 8.72673584])

# Predição de lambda incial oficial

In [3094]:
def initial_lambda_oficial(p):
    
    soma_a = 0
    soma_b = 0
    soma_d = 0
    slopes = []
    for i in range(p.n):
        soma_a += p.a[i]
        soma_b += p.b[i]
        soma_d += p.d[i]
        slopes.append((p.b[i]/p.d[i])*p.b[i])
        
    soma_a = soma_a/p.n
    soma_b = soma_b/p.n
    soma_d = soma_d/p.n
    rn = p.r/p.n
    return regr.predict(scaler.transform(np.asarray([[soma_a,soma_b,soma_d,r]])))[0],slopes,soma_a,soma_b,soma_d,rn

In [3095]:
def just_phi_oficial(p,lamb,r):
    phi = r * -1
    x = []
    
    for i in range(p.n):
        
        x.append( (p.b[i] * lamb + p.a[i])/p.d[i])

        if x[i] < p.low[i]:
            x[i] = p.low[i]
        elif x[i] > p.up[i]:
            x[i] = p.up[i]
        phi = phi + p.b[i] * x[i];
    return phi

In [3096]:
MAX_IT = 10
INFINITO_NEGATIVO = -999999999;
INFINITO_POSITIVO = 999999999;

def newton_new_lamb_init(p,lamb,phi,slopes):
    lambs = []
    alfa = INFINITO_NEGATIVO;
    beta = INFINITO_POSITIVO;
    phi_alfa = 0.0;
    phi_beta = 0.0;
    x = []
    r = p.r
    deriv = 0
    
    deriv, phi, x = phi_lambda(p,lamb,phi,deriv,slopes,r)
    lambs.append(lamb)
    it = 1

    while phi != 0.0 and it <= MAX_IT:
        if phi > 0:
#             print("positivo")
            beta = lamb
            lambda_n = 0.0
            if deriv > 0.0:
                
                lambda_n = lamb - (phi/deriv)
                if abs(lambda_n - lamb) <= 0.00000000001:
                    phi = 0.0
                    break
                if lambda_n > alfa:
                    lamb = lambda_n
                else:
#                     print("aqui")
                    phi_beta = phi;
#                     lamb = secant(p,x,alfa,beta,phi_alfa,phi_beta,r);
#             if deriv == 0.0:
#                 lamb = breakpoint_to_the_left(p,lamb);
#                 if lamb <= INFINITO_NEGATIVO or lamb >= INFINITO_POSITIVO:
#                     break
                
        else:
#             print("negativo")
            alfa = lamb;
            lambda_n = 0.0;

            if deriv > 0.0:
                lambda_n = lamb - (phi/deriv)
                if abs(lambda_n - lamb) <= 0.00000000001:
                    phi = 0.0
                    break
                
                if lambda_n < beta:
                    lamb = lambda_n
                else:
#                     print("aqui")
                    phi_alfa = phi;
#                     lamb = secant(p,x,alfa,beta,phi_alfa,phi_beta,r);
#             if deriv == 0.0:
#                 lamb = breakpoint_to_the_right(p,lamb)
#                 if lamb <= INFINITO_NEGATIVO or lamb >= INFINITO_POSITIVO:
#                     break
        
        
        deriv, phi, x = phi_lambda(p,lamb,phi,deriv,slopes,r)
        it = it + 1
        lambs.append(lamb)
        
    if phi == 0.0:
        return it,lambs
    elif alfa == beta:
        return -1,lambs
    else:
        return -2,lambs

In [3115]:
n = 100
p = generate_cqk_problem(n)

In [3116]:
%%time 

lambs11 = []
phis11 = []

lamb,slopes,soma_a,soma_b,soma_d,rn = initial_lambda_oficial(p)
lambs11.append(lamb)

phi_ficial = just_phi_oficial(p,lamb,p.r)
phis11.append(phi_ficial)

lamb = regr_second_lambda.predict(scaler_second.transform(np.asarray([[soma_a,soma_b,soma_d,rn,lamb,phi]])))[0]
lambs11.append(lamb)

phi_ficial = just_phi_oficial(p,lamb,p.r)
phis11.append(phi_ficial)

test = regr_third_lambda.predict(scaler_third.transform(np.asarray([[soma_a,soma_b,soma_d,rn,lambs11[0],phis11[0],lambs11[1],phis11[1]]])))
test = test[0]

print(lambs11,phis11,test)

phi_init = phis11[-1]



newton_new_lamb_init(p,test,phi_init,slopes)

[6.713950529945758, 8.199574407921991] [-1303.720727299362, -339.86251733151903] 9.231108188542343
CPU times: user 4.7 ms, sys: 2.73 ms, total: 7.42 ms
Wall time: 6.96 ms


(3, [9.231108188542343, 8.718883047771524, 8.755446022815812])

In [3117]:
%%time 
newton(p)

CPU times: user 550 µs, sys: 0 ns, total: 550 µs
Wall time: 555 µs


(4,
 [7.069643288517168, 8.514033349086409, 8.756860544740144, 8.755446022815823],
 [713.3417734668951, 626.0566614651611, 626.0566614651611, 626.0566614651611],
 [-1030.3437673844192,
  -152.0235834239253,
  0.8855708735096641,
  -1.4495071809506044e-12])

In [ ]:
tem que prever esse phi inicial pelo menos